Time series data consists of observations collected sequentially over time. It is used to analyze trends, seasonal patterns, and other time-dependent behaviors, and it's often employed in forecasting and other temporal analyses.



In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import seaborn as sns
import datetime as dt
from datetime import date, timedelta
import plotly.graph_objects as go
import plotly.express as px
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
import plotly.express as px
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose


In [34]:
# define the dates to fetch data
today = date.today()
d1= today.strftime("%Y-%m-%d")
end_date = d1
d2 = today - timedelta(days=365)
d2 = d2.strftime("%Y-%m-%d")
start_date = d2

# fetch the data from yahoo finance
ticker = "GOOGL"
df = yf.download(ticker, start_date, end_date, progress=False)
print(df.head())




In [35]:
from datetime import date, timedelta
# define the dates to fetch data
today = date.today()
d1 = today.strftime("%Y-%m-%d")
end_date = d1
d2 = date.today() - timedelta(days=730)
d2 = d2.strftime("%Y-%m-%d")
start_date = d2
print("data will be between these dates: ", start_date, "and", end_date)

In [36]:
# define ticker
ticker = 'GOOGL'
df = yf.download(ticker, start_date, end_date, progress=False)
print(df.head())


In [37]:
df['Date'] = df.index
df.reset_index(drop=True, inplace=True)

In [38]:
df.insert(0, 'Date', df.index, True)

In [39]:
df.info()

In [40]:
df["Date"] = df.index
    

In [41]:
# # select columns
# df = df[["Date", "Close"]]
# df.head()

In [42]:
#plot the data to see how it looks like
plt.figure(figsize=(16,8))
plt.plot(df["Date"], df["Close"])
plt.xlabel("Time")
plt.ylabel("Price")
plt.show()




In [43]:
df.info()


In [44]:
df.describe()

In [45]:
# stationarity check on data

from statsmodels.tsa.stattools import adfuller
result = adfuller(df["Close"])
print("ADF Statistic:", result[0])
print("p-value:", result[1])
print("Critical Values:")
for key, value in result[4].items():
    print("\t%s: %.3f" % (key, value))


Based on the ADF test results:

The ADF Statistic is -1.883, which is not lower than the critical values (e.g., -3.444 at 1%),
The p-value is 0.340 (above the common threshold of 0.05).
Thus, you cannot reject the null hypothesis of a unit root, and the data is not stationary.

In [46]:
# decompose the data to see the trend, seasonality and residuals
result = seasonal_decompose(df["Close"], model='multiplicative', period=30)
result.plot()
plt.show()

In [ ]:
# plot autocorrelation and partial autocorrelation
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
plot_acf(df["Close"], lags=30)
plot_pacf(df["Close"], lags=30)
plt.show()from statsmodels.tsa.stattools import adfuller

result = adfuller(df["Close"])
print("ADF Statistic: {:.3f}".format(result[0]))
print("p-value: {:.3f}".format(result[1]))
for key, value in result[4].items():
    print("\t{}: {:.3f}".format(key, value))    from statsmodels.tsa.stattools import adfuller
    
    result = adfuller(df["Close"])
    print("ADF Statistic: {:.3f}".format(result[0]))
    print("p-value: {:.3f}".format(result[1]))
    for key, value in result[4].items():
        print("\t{}: {:.3f}".format(key, value))        from statsmodels.tsa.stattools import adfuller
        
        result = adfuller(df["Close"])
        print("ADF Statistic: {:.3f}".format(result[0]))
        print("p-value: {:.3f}".format(result[1]))
        for key, value in result[4].items():
            print("\t{}: {:.3f}".format(key, value))            from statsmodels.tsa.stattools import adfuller
            
            result = adfuller(df["Close"])
            print("ADF Statistic: {:.3f}".format(result[0]))
            print("p-value: {:.3f}".format(result[1]))
            for key, value in result[4].items():
                print("\t{}: {:.3f}".format(key, value))

In [48]:
from statsmodels.tsa.stattools import adfuller

result = adfuller(df["Close"])
print("ADF Statistic: {:.3f}".format(result[0]))
print("p-value: {:.3f}".format(result[1]))
for key, value in result[4].items():
    print("\t{}: {:.3f}".format(key, value))

In [57]:
#  plots 
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
pd.plotting.autocorrelation_plot(df["Close"])
plot_acf(df["Close"], alpha=0.05) # type: ignore
from statsmodels.tsa.stattools import acf, pacf
x_acf = pd.DataFrame(acf(df["Close"]))
#x_acf.plot(kind='bar')
print(x_acf)

In [58]:
# lets find the q value
# paritial autocorrelation
from statsmodels.tsa.stattools import acf, pacf
x_acf = pd.DataFrame(acf(df["Close"])) # autocorrelation
x_pacf = pd.DataFrame(pacf(df["Close"])) # partial autocorrelation
print(x_pacf)


In [61]:
# New cell: Find the optimal d value (order of differencing)
from statsmodels.tsa.stattools import adfuller

def find_d(series):
    d = 0
    current_series = series.copy()
    while True:
        result = adfuller(current_series.dropna())
        if result[1] < 0.05:
            break
        current_series = current_series.diff().dropna()
        d += 1
    return d

d_value = find_d(df["Close"])
print("Optimal d value:", d_value)

In [ ]:
# p= o.340, d=1, q=2
# define the variable p, q, d
p = 0.340
d = 1
q = 2

In [62]:
# lets run arima with pmdarima
from pmdarima import auto_arima
model = auto_arima(df["Close"], start_p=0, start_q=0, max_p=5, max_q=5, m=12, start_P=0, seasonal=True, d=1, D=1, trace=True, error_action='ignore', suppress_warnings=True)


# Auto arima to find best value and best modelde for ARIMA

In [ ]:
# Auto arima to find best value and best modelde for ARIMA
#model= auto_arima(df["Close"], seasonal=True suppress_warnings=True)
# print the summary of the model
#print(model.summary()

In [63]:
# print model summary
print(model.summary())

In [66]:
# statsmodel and arima
from statsmodels.tsa.arima.model import ARIMA
p,d,q = 0,1,0
model = ARIMA(df["Close"], order=(p,d,q))
model_fit = model.fit()
print(model_fit.summary())

# predict next 30 days
forecast = model_fit.predict(steps=30)
print(forecast)


# plot the forecasted values
import matplotlib.pyplot as plt
plt.plot(df.index, df["Close"])
plt.plot(forecast.index, forecast)
plt.show()



In [67]:
# lets run sarimax with pmdarima
import statsmodels.api as sm
model = sm.tsa.statespace.SARIMAX(df["Close"], order=(p,d,q), seasonal_order=(p,d,q,12))
results = model.fit()
print(results.summary())




In [69]:
# plot the actual and predicted values of sarima model
plt.plot(df.index, df["Close"])
plt.plot(results.predict().index, results.predict())
plt.show()


# Hyper parameter tuninging

In [73]:
# hyperparameter tuning
from sklearn.metrics import mean_squared_error
import itertools
from statsmodels.tsa.statespace.sarimax import SARIMAX
p = d = q = range(0, 2)
# generate all different combinations of p, d and q triplets
pdq = list(itertools.product(p, d, q))
# generate all different combinations of seasonal p, q and q triplets and m quadruplets
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

best_aic = np.inf
best_pdq = None
best_seasonal_pdq = None
temp_model = None

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            temp_model = SARIMAX(df["Close"], order=param, seasonal_order=param_seasonal, enforce_stationarity=False, enforce_invertibility=False)
            results = temp_model.fit()
            if results.aic < best_aic:
                best_aic = results.aic
                best_pdq = param
                best_seasonal_pdq = param_seasonal
        except:
            continue
print("Best PDQ parameters:", best_pdq)
print("Best seasonal PDQ parameters:", best_seasonal_pdq)
print("Best AIC:", best_aic)


# predict the next 30 days
model = SARIMAX(df["Close"], order=best_pdq, seasonal_order=best_seasonal_pdq, enforce_stationarity=False, enforce_invertibility=False)
results = model.fit()
forecast = results.get_forecast(steps=30)
print(forecast.summary_frame())
print(forecast.predicted_mean)
# plot the forecasted values
plt.plot(df.index, df["Close"])
plt.plot(forecast.predicted_mean.index, forecast.predicted_mean)    
plt.show()
